## NOTE: 
it seems we won't be able to use chat templates, so no instruction fine tuning it seems

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoConfig, AutoModelForCausalLM, AutoTokenizer
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from typing import Dict, List
import argparse
from datasets import load_dataset

In [2]:
model_name = "Qwen/Qwen3-0.6B-Base"  # Replace with your checkpoint

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="mps",
    use_cache=True
).eval()

# prepare the model input
prompt = "Once upon a time, there was a Wait,"
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

Does Qwen do it like this?

```
<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
What is the capital of France?<|im_end|>
<|im_start|>assistant
The capital of France is Paris.<|im_end|>
```

Instead of having <|system|> being special tokens?


In [4]:
tokenizer.chat_template

'{%- if tools %}\n    {{- \'<|im_start|>system\\n\' }}\n    {%- if messages[0].role == \'system\' %}\n        {{- messages[0].content + \'\\n\\n\' }}\n    {%- endif %}\n    {{- "# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\\n" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}\n    {%- if messages[0].role == \'system\' %}\n        {{- \'<|im_start|>system\\n\' + messages[0].content + \'<|im_end|>\\n\' }}\n    {%- endif %}\n{%- endif %}\n{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}\n{%- for message in messa

In [5]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B")


In [7]:
tokenizer.special_tokens_map

{'eos_token': '<|im_end|>',
 'pad_token': '<|endoftext|>',
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [8]:
tokenizer.special_tokens_map_extended

{'eos_token': AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 'pad_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 'additional_special_tokens': ['<|im_start|>',
  '<|im_end|>',
  '<|object_ref_start|>',
  '<|object_ref_end|>',
  '<|box_start|>',
  '<|box_end|>',
  '<|quad_start|>',
  '<|quad_end|>',
  '<|vision_start|>',
  '<|vision_end|>',
  '<|vision_pad|>',
  '<|image_pad|>',
  '<|video_pad|>']}

In [9]:
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/mistral-7b-sft-beta")

tokenizer.special_tokens_map_extended

{'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 'pad_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
 'additional_special_tokens': ['<unk>', '<s>', '</s>']}

In [10]:
tokenizer.convert_tokens_to_ids('<s>')

1

In [11]:
tokens_assistant = tokenizer.tokenize('<|assistant|>')
print(f"Tokenizing '<|assistant|>' results in: {tokens_assistant}")
if len(tokens_assistant) == 1 and tokens_assistant[0] == '<|assistant|>':
    print("<|assistant|> is tokenized as a single special token.")
else:
    print("<|assistant|> is NOT tokenized as a single special token.")


Tokenizing '<|assistant|>' results in: ['▁<', '|', 'ass', 'istant', '|', '>']
<|assistant|> is NOT tokenized as a single special token.


In [12]:
example = {
  "messages": [
    {"role": "user", "content": "Provide a detailed analysis..."},
    {"role": "assistant", "content": "[Sports Commentary Script] ..."}
  ]
}

tokenized = tokenizer.apply_chat_template(example["messages"], tokenize=True, truncation=True, max_length=2048)

tokenized

[523,
 28766,
 1838,
 28766,
 28767,
 13,
 18325,
 547,
 264,
 10537,
 5643,
 1101,
 2,
 28705,
 13,
 28789,
 28766,
 489,
 11143,
 28766,
 28767,
 13,
 28792,
 28735,
 2729,
 19479,
 628,
 15548,
 28793,
 3850,
 2,
 28705,
 13]

In [13]:
def preprocess(example):
    tokens = tokenizer.apply_chat_template(
        example["messages"], tokenize=True, truncation=True, max_length=2048
    )
    return {"input_ids": tokens}

tokenized_dataset = dataset["train"].map(preprocess, remove_columns=["messages"])

NameError: name 'dataset' is not defined

In [2]:
from datasets import get_dataset_config_names, get_dataset_split_names, load_dataset

splits = get_dataset_split_names("nvidia/OpenCodeReasoning-2")
print("Available splits:", splits)

print("config names: ", get_dataset_config_names("nvidia/OpenCodeReasoning-2"))


Resolving data files:   0%|          | 0/70 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/59 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/70 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/59 [00:00<?, ?it/s]

Available splits: ['python', 'cpp']


Resolving data files:   0%|          | 0/70 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/59 [00:00<?, ?it/s]

config names:  ['train']


In [ ]:
dataset = load_dataset("nvidia/OpenCodeReasoning-2", name="train", split="train[:1%]") # to load just a part of the dataset

Resolving data files:   0%|          | 0/70 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/59 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/70 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/59 [00:00<?, ?it/s]

train-00001-of-00070.parquet:  86%|########5 | 315M/367M [00:00<?, ?B/s]

train-00002-of-00070.parquet:   0%|          | 0.00/365M [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset

# Load just 1% of the dataset by using the streaming mode and sampling
dataset = load_dataset(
    "your_dataset_name",  # Replace with actual dataset name
    split="train",  # Or whichever split you want
    streaming=True,
)

# Sample just 1% using the streaming API
sampled_dataset = dataset.shuffle(seed=42).take_percent(1)

# Convert the streaming dataset to a regular dataset in memory
# This downloads only the sampled 1%
sampled_dataset = sampled_dataset.map(lambda x: x, batched=False)

sampled_dataset.save_to_disk("my_1percent_dataset")